# 교차검증

- kfold
- stratified kfold
- cross_val_score
- cross_validate
- GridSearchCV

In [33]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
from sklearn.datasets import load_iris

iris=load_iris()

In [12]:
features=iris.data
target=iris.target

In [5]:
print('특성 shape :',features.shape)
print('타깃 shape :',target.shape)

특성 shape : (150, 4)
타깃 shape : (150,)


In [22]:
decisionTree=DecisionTreeClassifier(random_state=412)

In [60]:
kfold=KFold(n_splits=5)

cv_accuracy=list()

### kfold.split 을 이용하여 cv 하는 방법

In [61]:
n_iter=0

# KFold 객체의 split() 을 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환

for train_index, test_index in kfold.split(features):
    x_train, x_test=features[train_index], features[test_index]
    y_train, y_test=target[train_index], target[test_index]
    
    # 학습 및 예측
    
    decisionTree.fit(x_train, y_train)
    
    y_pred=decisionTree.predict(x_test)
    
    n_iter+=1
    
    # 반복시 마다 정확도 측정
    accuracy=np.round(accuracy_score(y_test, y_pred),4)
    
    train_size=x_train.shape[0]
    test_size=x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 : 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.9, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.8, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


In [62]:
print('## 교차 검증별 정확도 : ',cv_accuracy)

## 교차 검증별 정확도 :  [1.0, 1.0, 0.9, 0.9333, 0.8]


In [63]:
print('## 평균 검증 정확도 : ',np.mean(cv_accuracy))

## 평균 검증 정확도 :  0.92666


### cross_val_score 을 이용한 교차 검증

In [57]:
df_clf=DecisionTreeClassifier(random_state=412)
kfold=KFold(n_splits=5)

cv_results=cross_val_score(df_clf, features, target, cv=kfold, scoring='accuracy')

In [58]:
cv_results

array([1.        , 1.        , 0.9       , 0.93333333, 0.8       ])

In [59]:
cv_results.mean()

0.9266666666666665

### Stratified KFold

불균형한 데이터 분포를 가질 때, stratified kfold를 사용함

In [39]:
import pandas as pd

iris_df=pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [40]:
kfold=KFold(n_splits=3)

n_iter=0

for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    
    label_train=iris_df['label'].iloc[train_index]
    label_test=iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포: 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포: 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포: 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    50
Name: label, dtype: int64


In [41]:
from sklearn.model_selection import StratifiedKFold

stratified_kfold=StratifiedKFold(n_splits=3)

n_iter=0

for train_index, test_index in stratified_kfold.split(iris_df, iris_df['label']): # label 을 같이 넣어줘야 분포를 반영해줌
    n_iter+=1
    label_train=iris_df['label'].iloc[train_index]
    label_test=iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포: 
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포: 
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포: 
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    17
1    17
0    16
Name: label, dtype: int64


In [44]:
features=iris.data
target=iris.target

In [47]:
df_clf=DecisionTreeClassifier(random_state=0)

skfold=StratifiedKFold(n_splits=3)
n_iter=0

cv_accuracy=list()

for train_idx, test_idx in skfold.split(features, target):
    x_train, x_test= features[train_idx], features[test_idx]
    y_train, y_test= target[train_idx], target[test_idx]
    
    df_clf.fit(x_train, y_train)
    
    y_pred=df_clf.predict(x_test)
    
    n_iter+=1
    accuracy=np.round(accuracy_score(y_test, y_pred),4)
    
    train_size=x_train.shape[0]
    test_size=x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_idx))
    
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]


In [51]:
print('\n## 교차 검증별 정확도 : ', cv_accuracy)
print('## 평균 검증 정확도 : ', np.round(np.mean(cv_accuracy),4))


## 교차 검증별 정확도 :  [0.98, 0.94, 0.98]
## 평균 검증 정확도 :  0.9667


#### cross_val_score 을 사용한 stratified kfold

In [52]:
skfold=StratifiedKFold(n_splits=3)
df_clf=DecisionTreeClassifier()

cv_results=cross_val_score(df_clf, features, target, cv=skfold, scoring='accuracy')

In [53]:
print('\n## 교차 검증별 정확도 : ', cv_results)
print('## 평균 검증 정확도 : ', np.round(np.mean(cv_results),4))


## 교차 검증별 정확도 :  [0.98 0.94 0.98]
## 평균 검증 정확도 :  0.9667


일반적으로 분류(classification) 에서 교차검증을 할 때는 kfold 가 아닌 stratified kfold를 사용한다. 

회귀(regression) 에서는 결정값이 이산값 형태의 레이블이 아닌 연속된 숫자값이기 때문에 결정값의 분포를 정하는 의미가 없기 때문이다.

- cross_val_score 는 모델에 회귀 모델이 들어가면 cv를 kfold로, 분류 모델이 들어가면 cv를 stratified kfold로 자동 설정한다

#### cross_validate

cross_val_score는 단 하나의 평가지표를 사용했다면, cross_validate는 여러개의 평가지표를 반환 할 수 있다.

In [77]:
from sklearn.model_selection import cross_validate

dt_clf=DecisionTreeClassifier()
skfold=StratifiedKFold(n_splits=3)

cv_results=cross_validate(dt_clf, features, target, cv=skfold, scoring=['accuracy'])

In [78]:
cv_results

{'fit_time': array([0.00099897, 0.00099897, 0.00103903]),
 'score_time': array([0.00099492, 0.00099635, 0.        ]),
 'test_accuracy': array([0.98, 0.94, 0.98])}

In [82]:
np.round(np.mean(cv_results['test_accuracy']),4)

0.9667

#### GridSearchCV

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

x_train, x_test, y_train, y_test=train_test_split(features, target, test_size=0.2, random_state=412)

dtree=DecisionTreeClassifier()

parameters={'max_depth':[1,2,3], 'min_samples_split':[2,3]}

grid_tree=GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

 - refit : default 가 True 이며 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습 시킨다

In [88]:
grid_tree.fit(x_train, y_train)

scores_df=pd.DataFrame(grid_tree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.666667,5,0.65,0.675,0.675
1,"{'max_depth': 1, 'min_samples_split': 3}",0.666667,5,0.65,0.675,0.675
2,"{'max_depth': 2, 'min_samples_split': 2}",0.941667,1,0.95,0.950,0.925
3,"{'max_depth': 2, 'min_samples_split': 3}",0.941667,1,0.95,0.950,0.925
4,"{'max_depth': 3, 'min_samples_split': 2}",0.933333,3,0.95,0.950,0.900
5,"{'max_depth': 3, 'min_samples_split': 3}",0.933333,3,0.95,0.950,0.900


In [91]:
scores_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_min_samples_split', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'],
      dtype='object')

In [92]:
print('GridSearchCV 최적 파라미터 : ', grid_tree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_tree.best_score_))

GridSearchCV 최적 파라미터 :  {'max_depth': 2, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9417


In [93]:
# GridSearchCV의 refit 으로 이미 학습된 estimator 반환

estimator=grid_tree.best_estimator_

pred=estimator.predict(x_test)

print('테스트 데이터 세트 정확도 : ',np.round(accuracy_score(y_test,pred),4))

테스트 데이터 세트 정확도 :  0.9333
